# Parallel Programming

### 병렬 프로그래밍(parallel programming)은 계산을 여러 부분으로 나누어 계산하여도 될 때, 이를 병렬화하여 여러개의 thread에 할당하여 동시에 작업을 수행하는 테크닉이다.

1. 병렬화하여 수행할 function_to_apply 함수를 설정한다. 이 때, 일반적인 함수처럼 return을 사용하지 않고, output이라는 아규먼트를 받아서 output.put(result)를 이용하여 결과를 반환한다.
2. original_data는 원본 데이터이다. np.array_split(original_data, n_threads)는 original_data를n_threads 개수 만큼 균일하게 나누어준다. 각각 나누어진 데이터에 function_to_apply 를 수행할 것이다.
3. Queue()를 n_threads 개수만큼 생성한다.
4. Process() 에 수행할 작업인 function_to_apply 를 할당하고 데이터와 생성된 Queue()를 함수의 파라미터로 넣어준다.
5. 생성된 Process().start()를 시작한다.
6. 작업이 완료되면 Process().get()를 통하여 작업결과를 반환받는다.
7. 생성된 큐와 프로세스를 Queue().close()와 Process().join()를 통하여 닫아준다
8. np.concatenate(results)를 통하여 나누어서 계산한 결과를 병합한다.

In [3]:
import numpy as np
from multiprocessing import Process, Queue
 
n_threads = 40
 
splitted = np.array_split(original_data, n_threads)
 
def function_to_apply(a, output):
    applied = do_function(a)
    output.put(applied)
 
if __name__ == '__main__':
    outputs = []
    procs = []
    print "append process"
    for i in splitted:
        print "hello queue!"
        output = Queue()
        outputs.append(output)
    for i, data_i in enumerate(splitted):
        procs.append(Process(target=function_to_apply, args=(data_i, outputs[i])))
    print "process starts"
    for p in procs:
        p.start()
    results = []
    print procs
    print "append outputs"
    print outputs
    for output in outputs:
        result = output.get()
        results.append(result)
    for output in outputs:
        output.close()
        print "output end!"
    print "finish process"
    for p in procs:
        p.join()
 
results = np.concatenate(results)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(outputs)? (<ipython-input-3-baea0f29edd3>, line 28)

### Pandas 데이터프레임의 경우는 다음과 같이 병렬화할 수 있다.

In [4]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
 
result_df = parallelize_dataframe(df, function_to_apply)

NameError: name 'df' is not defined

#### 출처: CSBL Wiki (Ingoo Lee 2018/04/17 15:12)